# 2020_10_15:Dimensionality Reduction Methods



FAILED using TruncatedSVD/LatentSentimentAnalysis. Maximum dimension achieved before 32GB RAM ceiling = 400. Maximum training F1 score ~ 47%.

Going for semiautomatic chi-square based analysis.

# Imports

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt  
import wordcloud as wc 
import seaborn as sns 
import nltk
import re
from tqdm import tqdm
tqdm.pandas()

train_dset_df = pd.read_csv("./dataset/pp/2020_10_04_preprocessed_train.csv")
test_dset_df = pd.read_csv("./dataset/pp/2020_10_04_preprocessed_test.csv")
train_dset_df["preprocessed"] = train_dset_df["preprocessed"].progress_apply(eval)
test_dset_df["preprocessed"] = test_dset_df["preprocessed"].progress_apply(eval)
train_dset_df = train_dset_df.drop(axis="columns", labels="Unnamed: 0")
test_dset_df = test_dset_df.drop(axis="columns", labels="Unnamed: 0")
from tqdm import tqdm 
tqdm.pandas()
import gc

100%|██████████| 522449/522449 [00:06<00:00, 78500.58it/s]


# Snowball Stemming

In [2]:
from nltk.stem import SnowballStemmer

In [3]:
snow_stem = SnowballStemmer("english")

In [4]:
train_dset_df["preprocessed"] = train_dset_df["preprocessed"].progress_apply(lambda x: [snow_stem.stem(i) for i in x])

100%|██████████| 783673/783673 [00:53<00:00, 14631.65it/s]


In [5]:
test_dset_df["preprocessed"] = test_dset_df["preprocessed"].progress_apply(lambda x: [snow_stem.stem(i) for i in x])

100%|██████████| 522449/522449 [00:34<00:00, 15272.64it/s]


In [6]:
train_dset_df["preprocessed"] = train_dset_df["preprocessed"].progress_apply(lambda x: " ".join(x))

100%|██████████| 783673/783673 [00:00<00:00, 1178633.61it/s]


In [7]:
test_dset_df["preprocessed"] = test_dset_df["preprocessed"].progress_apply(lambda x: " ".join(x))

100%|██████████| 522449/522449 [00:00<00:00, 1157020.44it/s]


In [8]:
train_dset_questions = train_dset_df["preprocessed"]
test_dset_questions = test_dset_df["preprocessed"]

questions = pd.concat([train_dset_questions, test_dset_questions])

# TFIDFVectorizer

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [35]:
vectorizer = CountVectorizer()

In [36]:
vectorizer.fit(train_dset_questions)

CountVectorizer()

In [37]:
sparse_x_train = vectorizer.transform(train_dset_df["preprocessed"])

In [38]:
sparse_x_train

<783673x69405 sparse matrix of type '<class 'numpy.int64'>'
	with 5499037 stored elements in Compressed Sparse Row format>

# Dimensionality reduction MK II

In [39]:
sparse_x_all = vectorizer.transform(questions)

In [40]:
sparse_x_train.shape

(783673, 69405)

In [41]:
y = train_dset_df["target"].to_numpy()

In [42]:
curr_column  = sparse_x_train[:,0].toarray().reshape(-1, )
curr_column = (curr_column > 0).astype(int)

In [43]:
y.shape

(783673,)

In [44]:
curr_column.shape

(783673,)

In [45]:
n_curr_col_one = curr_column.sum()
n_curr_col_one

7

In [47]:
n_curr_col_zero = curr_column.shape[0] - n_curr_col_one
n_curr_col_zero

783666

In [49]:
n_y_and_curr_col = np.sum(np.logical_and(curr_column, y).astype(int))
n_y_and_curr_col

0

In [51]:
n_y_and_not_curr_col = np.sum(np.logical_and(1-curr_column, y).astype(int))
n_y_and_not_curr_col

48451

In [52]:
p_y_if_curr_col = n_y_and_curr_col/n_curr_col_one
p_y_if_curr_col

0.0

In [53]:
p_y_if_not_curr_col = n_y_and_not_curr_col/n_curr_col_zero
p_y_if_not_curr_col

0.06182608407153047

In [55]:
p_y = np.sum(y)/y.shape[0]
p_y

0.061825531822584164

In [32]:
# from sklearn.decomposition import TruncatedSVD

In [69]:
# tsvd = TruncatedSVD(n_components = 400, random_state=42)

In [34]:
# tsvd.fit(sparse_x_all)

TruncatedSVD(n_components=400, random_state=42)

# Training

In [35]:
sparse_x_train = vectorizer.transform(train_dset_df["preprocessed"])
# transformed_x_train = tsvd.transform(sparse_x_train)

In [36]:
# transformed_x_train.shape

(783673, 400)

In [37]:
# from sklearn.kernel_approximation import Nystroem
from sklearn.svm import LinearSVC

In [58]:
svm = LinearSVC(verbose=2, max_iter = 4000, class_weight= {0:1, 1:})

In [59]:
svm.fit(sparse_x_train, train_dset_df["target"])

[LibLinear]

LinearSVC(class_weight={0: 1, 1: 8}, max_iter=4000, verbose=2)

In [60]:
yhat = svm.predict(sparse_x_train)

In [61]:
train_dset_df["yhat"] = yhat

In [62]:
from sklearn.metrics import f1_score

In [63]:
wrongs = train_dset_df[train_dset_df["yhat"] != train_dset_df["target"]]

In [64]:
y = train_dset_df["target"]

In [65]:
f1_score(y, yhat)

0.4738771575532835

In [66]:
[(name, len(x)) for name, x in wrongs.groupby(by="target")]

[(0, 59993), (1, 14778)]

In [67]:
[(name, len(x)) for name, x in train_dset_df.groupby(by="target")]

[(0, 735222), (1, 48451)]

# Testing and writing output

In [48]:
sparse_x = vectorizer.transform(test_dset_df["preprocessed"])

In [49]:
yhat = svm.predict(sparse_x)

In [50]:
test_dset_df.question_text = yhat

In [51]:
test_dset_df = test_dset_df.drop(axis="columns", labels="preprocessed")
test_dset_df.head()

,qid,question_text
0,f56a9a31974dc66186e8,0
1,d957c3758060f45da303,0
2,ad822d5abaedb9e247b9,0
3,4e979c23eeb6a4bd1f2e,0
4,333cc031262566b8da49,0


In [52]:
test_dset_df = test_dset_df.rename(columns={"qid":"qid", "question_text":"target"})
test_dset_df.target = test_dset_df.target.apply(round)

In [53]:
test_dset_df.to_csv("./outputs/2020_10_15_b_testset_output.csv", index=False)